# Model Training and Evaluation

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/final.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges,age_group,bmi_category
0,19,female,27.900,0,yes,southwest,16884.92400,19-36,Overweight
1,18,male,33.770,1,no,southeast,1725.55230,0-18,Obese I
2,28,male,33.000,3,no,southeast,4449.46200,19-36,Obese I
3,33,male,22.705,0,no,northwest,21984.47061,19-36,Normal
4,32,male,28.880,0,no,northwest,3866.85520,19-36,Overweight


In [3]:
df = df.drop(columns=['age_group','bmi_category'])

In [4]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df_dummies = pd.get_dummies(df, drop_first=True)
df_dummies.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


In [6]:
X = df_dummies.drop('charges', axis=1)
y = df_dummies['charges']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [8]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [9]:
def evaluate(y_true, y_pred, name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name}:")
    print(f"  R²   = {r2:.4f}")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}\n")

In [10]:
model_lr = LinearRegression()
model_lr.fit(X_train_sc, y_train)

LinearRegression()

In [11]:
y_pred_lr = model_lr.predict(X_test_sc)

In [12]:
evaluate( y_test, y_pred_lr,"LinearRegression")

LinearRegression:
  R²   = 0.8069
  MAE  = 4177.05
  RMSE = 5956.34



In [13]:
model_rf = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=3, random_state=42)
model_rf.fit(X_train_sc, y_train)

RandomForestRegressor(max_depth=4, min_samples_leaf=3, random_state=42)

In [14]:
y_pred_rf = model_rf.predict(X_test_sc)

In [15]:
evaluate( y_test,y_pred_rf, "RandomForestRegressor")

RandomForestRegressor:
  R²   = 0.9022
  MAE  = 2452.87
  RMSE = 4239.18



In [16]:
model_gb = GradientBoostingRegressor(n_estimators=100, max_depth=4, max_leaf_nodes=3)
model_gb.fit(X_train_sc, y_train)

GradientBoostingRegressor(max_depth=4, max_leaf_nodes=3)

In [17]:
y_pref_gb = model_gb.predict(X_test_sc)

In [18]:
evaluate(y_test, y_pref_gb, "GradientBoostingRegressor")

GradientBoostingRegressor:
  R²   = 0.9007
  MAE  = 2529.47
  RMSE = 4272.40



### **Hyper Parameter Optimization**

In [59]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None,4, 5,6, 18,24],
    'min_samples_split': [2, 3,5],
    'min_samples_leaf': [1, 2, 3,5,4,6],
    'bootstrap': [True, False]
}

In [62]:
random_search_rf = RandomizedSearchCV(RandomForestRegressor(), param_grid, cv=5, n_iter = 50,random_state=42)
random_search_rf.fit(X_train_sc, y_train)

print(random_search_rf.best_estimator_)

RandomForestRegressor(max_depth=4, min_samples_leaf=3)


In [43]:
model_rf_best = RandomForestRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)
model_rf_best.fit(X_train_sc, y_train)

RandomForestRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)

In [44]:
y_pred_rf_best = model_rf_best.predict(X_test_sc)

In [45]:
evaluate(y_test, y_pred_rf_best, "RandomForestRegressor")

RandomForestRegressor:
  R²   = 0.9027
  MAE  = 2446.69
  RMSE = 4228.97



In [52]:
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None,4, 5,6, 18,24],
    'min_samples_split': [2, 3,5],
    'min_samples_leaf': [1, 2, 3,4,6],
}

In [53]:
random_search_gb = RandomizedSearchCV(GradientBoostingRegressor(), param_grid_gb, cv=5, n_iter = 50)
random_search_gb.fit(X_train_sc, y_train)

print(random_search_gb.best_estimator_)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)


In [55]:
model_gb_best = GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)
model_gb_best.fit(X_train_sc, y_train)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)

In [56]:
y_pred_gb_best = model_rf_best.predict(X_test_sc)

In [58]:
evaluate(y_test, y_pred_gb_best, "GradientBoostingRegressor")

GradientBoostingRegressor:
  R²   = 0.9027
  MAE  = 2446.69
  RMSE = 4228.97

